<a href="https://colab.research.google.com/github/taaha3244/HuggingFace-NLP/blob/main/FastApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi Pillow python-multipart torch transformers uvicorn nest-asyncio pyngrok

In [4]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")



Predicted answer: brown and black


In [12]:
# prepare image + question
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

text = "What are the colors of the cats?"

# prepare inputs
encoding = processor(image, text, return_tensors="pt")


In [14]:

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()


print("Predicted answer:", model.config.id2label[idx])

# TODO: put above code into a function that accepts image and text as input

Predicted answer: brown and black


In [15]:
def model_pipeline(text: str, image: Image):
    # prepare inputs
    encoding = processor(image, text, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()

    return  model.config.id2label[idx]

In [7]:
from typing import Union
from fastapi import FastAPI, UploadFile
import io
from PIL import Image

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.post("/ask")
def ask(text: str, image: UploadFile):
    content = image.file.read()

    image = Image.open(io.BytesIO(content))
    # image = Image.open(image.file)

    result = model_pipeline(text, image)
    return {"answer": result}






In [16]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


ngrok.set_auth_token("2bURGYstXHInKiMLzZWq2VvjUfB_4bdGXebAWXT8KjtiNpHUo")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://6cb0-34-141-197-179.ngrok-free.app


INFO:     Started server process [238]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [238]


KeyboardInterrupt: 